<a href="https://colab.research.google.com/github/wangqianyang/colab/blob/main/%E8%8B%B1%E8%AF%AD%E9%9F%B3%E9%A2%91%E7%94%9F%E6%88%90%E5%AD%97%E5%B9%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 步骤一：生成SRT字幕

In [6]:
# 安装 faster-whisper（更快、更省显存）和 ffmpeg
!pip install -q faster-whisper
!apt-get install -y ffmpeg -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 110.1 MB/s eta 0:00:00


### 上传文件

In [11]:
from google.colab import files

# 上传你的音频文件（支持 mp3, wav, m4a, mp4 等常见格式）
uploaded = files.upload()

# 获取上传的文件名（假设只上传一个音频文件）
audio_file = list(uploaded.keys())[0]
print(f"已上传音频文件：{audio_file}")

Saving 001.mp3 to 001.mp3
已上传音频文件：001.mp3


### 加载模型并转录，生成 SRT 字幕

In [12]:
from faster_whisper import WhisperModel
import datetime
import os

# 模型设置（如果 large-v3 显存不够，改成 "medium"）
model_size = "large-v3"  # 试试 "medium" 或 "small" 如果报错

model = WhisperModel(model_size, device="cuda", compute_type="float16")

# 关键修改：关闭 VAD 过滤 + 自动检测语言 + 降低阈值
segments_iterator, info = model.transcribe(
    audio_file,
    language=None,          # 自动检测语言（很重要！）
    beam_size=5,
    vad_filter=False,       # 先关闭 VAD，避免过度过滤
    vad_parameters=dict(min_silence_duration_ms=500),  # 即使开 VAD 也放宽
    word_timestamps=True,
    temperature=0.0,        # 更确定性输出
    no_speech_threshold=0.6,  # 默认 0.6，调低到 0.4 可以更敏感（如果仍空再试）
)

# 将 segments 迭代器转换为列表，以便多次使用
segments = list(segments_iterator)

# === 诊断信息输出 ===
print(f"检测到语言: {info.language} (概率: {info.language_probability:.2f})")
print(f"转录时长: {info.duration:.2f} 秒")
print(f"检测到段落数量: {len(segments)}") # 使用已转换为列表的 segments

if info.language_probability < 0.5:
    print("警告：语言检测置信度低，可能是非语音内容或噪音。")

if len(segments) == 0: # 使用已转换为列表的 segments
    print("错误：没有检测到任何语音段落！可能原因：")
    print("  - 音频无声或只有背景音乐")
    print("  - 文件损坏")
    print("  - 需要更低的 no_speech_threshold（试试 0.4）")
    print("  - 尝试关闭 vad_filter 或换小模型")
else:
    print("成功检测到语音！")

# 如果有段落，才生成 SRT
if len(segments) > 0: # 使用已转换为列表的 segments
    def format_timestamp(seconds: float):
        td = datetime.timedelta(seconds=seconds)
        hours, remainder = divmod(td.total_seconds(), 3600)
        minutes, seconds = divmod(remainder, 60)
        milliseconds = int((seconds - int(seconds)) * 1000)
        return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"

    srt_content = ""
    for i, segment in enumerate(segments, start=1): # 使用已转换为列表的 segments
        start_time = format_timestamp(segment.start)
        end_time = format_timestamp(segment.end)
        text = segment.text.strip()
        srt_content += f"{i}\n{start_time} --> {end_time}\n{text}\n\n"

    srt_filename = os.path.splitext(audio_file)[0] + ".srt"
    with open(srt_filename, "w", encoding="utf-8") as f:
        f.write(srt_content)

    print(f"SRT 字幕已保存为：{srt_filename}")
    files.download(srt_filename)
else:
    print("由于没有检测到语音，SRT 文件未生成。")

# 始终预览诊断（即使为空）
print("\n=== 额外调试：尝试打印原始 info ===")
print(info)


检测到语言: en (概率: 1.00)
转录时长: 40.83 秒
检测到段落数量: 8
成功检测到语音！
SRT 字幕已保存为：001.srt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


=== 额外调试：尝试打印原始 info ===
TranscriptionInfo(language='en', language_probability=0.9990234375, duration=40.829375, duration_after_vad=40.829375, all_language_probs=[('en', 0.9990234375), ('de', 0.00010466575622558594), ('es', 9.828805923461914e-05), ('ko', 7.134675979614258e-05), ('ja', 6.347894668579102e-05), ('vi', 5.692243576049805e-05), ('ru', 4.8279762268066406e-05), ('fr', 4.571676254272461e-05), ('pt', 3.3974647521972656e-05), ('zh', 2.9265880584716797e-05), ('tr', 2.6047229766845703e-05), ('nn', 2.5451183319091797e-05), ('pl', 2.5272369384765625e-05), ('cy', 2.3186206817626953e-05), ('it', 1.8477439880371094e-05), ('sv', 1.7881393432617188e-05), ('la', 9.298324584960938e-06), ('id', 7.331371307373047e-06), ('km', 6.735324859619141e-06), ('ar', 4.887580871582031e-06), ('th', 4.708766937255859e-06), ('nl', 4.231929779052734e-06), ('hu', 4.172325134277344e-06), ('fi', 3.874301910400391e-06), ('hi', 3.814697265625e-06), ('tl', 3.635883331298828e-06), ('ro', 2.682209014892578e-06), (

### （可选）：预览前几行字幕

In [13]:
# 打印前 10 段字幕预览
print("\n=== 前 10 段字幕预览 ===\n")
preview_segments = list(segments)[:10]
for i, segment in enumerate(preview_segments, start=1):
    print(f"{i}\n{format_timestamp(segment.start)} --> {format_timestamp(segment.end)}\n{segment.text.strip()}\n")


=== 前 10 段字幕预览 ===

1
00:00:00,000 --> 00:00:03,660
Hey, everyone. Welcome back to Real Life English Unleashed.

2
00:00:04,059 --> 00:00:07,360
I'm Maddy Evans, your favorite blonde chaos coordinator.

3
00:00:08,099 --> 00:00:13,019
And I'm Jake Harlan, professional airport sufferer and frequent flyer of regret.

4
00:00:13,640 --> 00:00:17,260
Maddy, why are we doing an entire episode on airports today?

5
00:00:17,960 --> 00:00:22,100
Because, Jake, airports are where English gets real, fast.

6
00:00:22,640 --> 00:00:28,559
You're stressed, tired, hungry, and suddenly you have to talk to 10 different people in one hour.

7
00:00:28,559 --> 00:00:36,299
Today, we're breaking down the top 10 airport moments with natural dialogues and all the key phrases you actually need.

8
00:00:37,119 --> 00:00:40,640
Perfect. I've got stories for days. Let's jump in.



## 步骤二：AI识别重点并生成双语+注解

In [14]:
# Cell 1: 安装必要库（只需运行一次）
!pip install -q openai pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [15]:
# Cell 2: 上传或加载你的 SRT 文件（如果已经上传过，可跳过上传直接指定文件名）
from google.colab import files
import pysrt
import os

# 如果你已经运行过第一步，SRT 文件应该已经在 Colab 环境中
# 直接指定文件名（替换成你的实际 SRT 文件名，例如 "your_audio.srt"）
srt_filename = "001.srt"  # <--- 这里改成你的 SRT 文件名 !!!

# 如果文件不在，直接上传
if not os.path.exists(srt_filename):
    print("文件不存在，请上传 SRT 文件")
    uploaded = files.upload()
    srt_filename = list(uploaded.keys())[0]

# 加载 SRT 并提取纯文本句子（按段落顺序）
subs = pysrt.open(srt_filename, encoding='utf-8')

# 提取每段的纯文本（去掉时间戳，只保留文字）
sentences = [sub.text.strip().replace('\n', ' ') for sub in subs if sub.text.strip()]

print(f"成功加载 SRT，共 {len(sentences)} 段句子")
print("\n前 10 句预览：")
for i, sent in enumerate(sentences[:10], 1):
    print(f"{i}: {sent}")

成功加载 SRT，共 8 段句子

前 10 句预览：
1: Hey, everyone. Welcome back to Real Life English Unleashed.
2: I'm Maddy Evans, your favorite blonde chaos coordinator.
3: And I'm Jake Harlan, professional airport sufferer and frequent flyer of regret.
4: Maddy, why are we doing an entire episode on airports today?
5: Because, Jake, airports are where English gets real, fast.
6: You're stressed, tired, hungry, and suddenly you have to talk to 10 different people in one hour.
7: Today, we're breaking down the top 10 airport moments with natural dialogues and all the key phrases you actually need.
8: Perfect. I've got stories for days. Let's jump in.


### 使用 OpenRouter GPT-OSS 模型 识别重点单词和词组

#### 1. 设置 OpenRouter API Key

您需要一个 OpenRouter API 密钥。如果还没有，请前往 [https://openrouter.ai/keys](https://openrouter.ai/keys) 创建一个。
在 Colab 中，通过左侧面板的“🔑”图标将密钥添加到 Secrets Manager。将其命名为 `OPENROUTER_API_KEY`。然后代码将自动获取该密钥。


In [23]:
import getpass
from google.colab import userdata

# 获取 Colab Secrets 中保存的 OPENROUTER_API_KEY
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

if not OPENROUTER_API_KEY:
    print("警告：未找到 OPENROUTER_API_KEY。请确保您已在 Colab Secrets 中设置它。")
else:
    print("OpenRouter API Key 已设置")


OpenRouter API Key 已设置


In [25]:
# Cell 4: 定义 Prompt 模板 + 批量调用 OpenRouter API
import json
import time
from openai import OpenAI # OpenRouter 兼容 OpenAI API

# 使用 OpenAI 兼容格式调用 OpenRouter API
client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# OpenRouter 模型名称
model_name = "openai/gpt-oss-120b:free"

# Prompt 模板（严格参考 BBC / EnglishClass101 风格）
prompt_template = """
你是英语学习字幕专家，参考BBC Learning English、English Addict with Mr Steve和EnglishClass101的字幕风格。
为每句英文输出：
1. 英文原文：重点单词/短语用[HL:yellow]标记（生词/高频新词），习语/搭配用[HL:red]标记。
2. 中文翻译：自然流畅、适合学习者。
3. 注解行（可选）：仅对重点词添加简短括号解释，如 (run into = 偶遇，meet by chance)。注解要极简，只出现必要时。

输出严格为 JSON 数组（无需其他文字），每个对象对应一句：
[
  {{
    "english": "标记后的英文",
    "chinese": "中文翻译",
    "annotation": "注解（如果没有则为空字符串 ""）"
  }},
  ...
]

现在处理以下句子（每句用数字编号）：
{numbered_sentences}

只输出 JSON！
"""

# 批量大小（建议 8-12 句一批，避免超过 token 限制）
batch_size = 10

enhanced_data = []

print("开始批量调用 OpenRouter API 处理...")
for i in range(0, len(sentences), batch_size):
    batch = sentences[i:i+batch_size]

    # 编号显示
    numbered = "\n".join(f"{j+1}: {sent}" for j, sent in enumerate(batch))

    prompt = prompt_template.format(numbered_sentences=numbered)

    print(f"处理第 {i+1}-{min(i+batch_size, len(sentences))} 句（共 {len(batch)} 句）...")

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,      # 低温度更稳定
            max_tokens=4096
        )

        content = response.choices[0].message.content.strip()

        # 尝试解析 JSON
        try:
            batch_json = json.loads(content)
        except json.JSONDecodeError:
            print("JSON 解析失败，原始输出：")
            print(content)
            print("请手动修正或重新运行本批")
            continue

        enhanced_data.extend(batch_json)

    except Exception as e:
        print(f"API 调用出错：{e}")
        print("可能原因：API Key 无效、配额不足、网络问题")
        break

    # 礼貌等待，避免触发限流
    time.sleep(2)

print(f"\n处理完成！共增强 {len(enhanced_data)} 段")


开始批量调用 OpenRouter API 处理...
处理第 1-8 句（共 8 句）...

处理完成！共增强 8 段


In [26]:
# Cell 5: 保存增强 JSON 并下载
import json
from google.colab import files

json_filename = os.path.splitext(srt_filename)[0] + "_enhanced.json"

with open(json_filename, "w", encoding="utf-8") as f:
    json.dump(enhanced_data, f, ensure_ascii=False, indent=2)

print(f"增强 JSON 已保存为：{json_filename}")
print("\n前 3 条预览：")
print(json.dumps(enhanced_data[:3], ensure_ascii=False, indent=2))

# 自动下载
files.download(json_filename)

增强 JSON 已保存为：001_enhanced.json

前 3 条预览：
[
  {
    "english": "Hey, everyone. [HL:red]Welcome back[/HL] to Real Life English Unleashed.",
    "chinese": "大家好，欢迎回到《Real Life English Unleashed》。",
    "annotation": "(Welcome back = 再次欢迎)"
  },
  {
    "english": "I'm Maddy Evans, your favorite blonde [HL:red]chaos coordinator[/HL].",
    "chinese": "我是 Maddy Evans，你们最喜欢的金发混乱协调员。",
    "annotation": "(chaos coordinator = 混乱协调员，搞笑自称)"
  },
  {
    "english": "And I'm Jake Harlan, [HL:yellow]professional airport sufferer[/HL] and [HL:red]frequent flyer[/HL] of regret.",
    "chinese": "我是 Jake Harlan，专业的机场受苦者，也是后悔的常旅客。",
    "annotation": "(professional airport sufferer = 专业的机场受苦者；frequent flyer = 常旅客)"
  }
]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 步骤三：自动生成ASS字幕（全自动，Python脚本）

In [29]:
import pysrt
import json
import re

# 加载原SRT（时间轴）
subs = pysrt.open(srt_filename)

# 加载AI处理的JSON
with open(json_filename, 'r', encoding='utf-8') as f:
    enhanced = json.load(f)

# ASS颜色定义
colors = {
    'yellow': '&H00FFFF&',   # ASS黄色
    'orange': '&H00A5FF&',
    'red': '&H0000FF&',
    'white': '&HFFFFFF&'
}

def add_color(text):
    # 替换[HL:color]为ASS标签
    def repl(match):
        word = match.group(1)
        color = colors.get(match.group(2), colors['white'])
        return f'{{\\c{color}}}{word}{{\\c{colors["white"]}}}'
    return re.sub(r'\[HL:(\w+?)\](.*?)\[/HL\]', repl, text)

# 合并
for i, sub in enumerate(subs):
    item = enhanced[i]
    english = add_color(item['english'])
    chinese = item['chinese']
    annotation = item.get('annotation', '')
    # ASS三行布局
    sub.text = f"{english}\\N{{\\fs20}}{chinese}\\N{{\\fs18\\c&H808080&}}{annotation}"

# 保存ASS
with open('final.ass', 'w', encoding='utf-8') as f:
    f.write('[Script Info]\nScriptType: v4.00+\n\n[V4+ Styles]\nFormat: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding\nStyle: Default,Arial,28,&H00FFFFFF,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,2,2,10,10,30,1\n\n[Events]\nFormat: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text\n')
    for sub in subs:
        f.write(f'Dialogue: 0,{sub.start},{sub.end},Default,,0,0,0,,{sub.text}\n')

print("ASS字幕生成完成！")

ASS字幕生成完成！


## 【调试用，不执行】下载youtube的音频试试

In [3]:
# 安装 yt-dlp
!pip install -q yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 111.8 MB/s eta 0:00:00


In [4]:
import yt_dlp
import os

def download_youtube_audio(youtube_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': '%(title)s.%(ext)s', # Output filename template
        'quiet': True, # Suppress console output for cleaner execution
        'no_warnings': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(youtube_url, download=True)
            filename = ydl.prepare_filename(info_dict)
            # Change extension to mp3 explicitly
            base, ext = os.path.splitext(filename)
            mp3_filename = base + '.mp3'
            print(f"成功下载音频：{mp3_filename}")
            return mp3_filename
    except Exception as e:
        print(f"下载失败：{e}")
        return None

# 示例用法：
# youtube_url = 'YOUR_YOUTUBE_URL_HERE'
# downloaded_file = download_youtube_audio(youtube_url)
# if downloaded_file:
#     print(f"文件已保存到：{downloaded_file}")


In [5]:
youtube_url = 'https://www.youtube.com/watch?v=gHvBLFYNzMc'
downloaded_file = download_youtube_audio(youtube_url)
if downloaded_file:
    print(f"文件已保存到：{downloaded_file}")

成功下载音频：Learn English with How I Met Your Mother (Barney's Look-Alike).mp3
文件已保存到：Learn English with How I Met Your Mother (Barney's Look-Alike).mp3


现在你可以使用上面的 `download_youtube_audio` 函数来下载YouTube视频的音频。只需将你想要下载的YouTube链接替换掉 `YOUR_YOUTUBE_URL_HERE` 即可。